In [1]:
import pandas as pd
from data_validation import data_prep, recoveries_prep, merge_recoveries, staging_map, date_cleaner
from functools import partial
from matrix_functions import base_matrices, absorbing_state, extract_pds, cure_rate
import plotly.graph_objs as go
import plotly.offline as pyo
    

# data = pd.read_csv('IFRS 9 PD Test Data.csv', dtype='object')
# data_rr = pd.read_csv("IFRS 9 Recoveries Test Data.csv", dtype='object')
collateral_data = pd.read_csv("Collateral Parameters.csv")
loan_book = pd.read_csv("IFRS 9 Loan Book Test Data.csv")

#### Run PD Model

In [2]:
# pd_data = data_prep(data, 3, 3)[0]
# rr_data = recoveries_prep(data_rr, 3)[0]
# df = merge_recoveries(pd_data, rr_data)
# matrices = base_matrices(df)
# absorbing_state(matrices, 3)
# cr_rr = cure_rate(df)
# final_output = extract_pds(matrices, 3)

In [3]:
staging_map_partial = partial(staging_map, matrix_size=3)
loan_book['staging'] = loan_book['days_past_due'].map(staging_map_partial)
date_cleaner(loan_book, 'report_date', 'clean_report_date')
date_cleaner(loan_book, 'disbursement_date', 'clean_disbursement_date')
date_cleaner(loan_book, 'maturity_date', 'clean_maturity_date')
loan_book.head()

,report_date,account_no,client_id,disbursement_date,maturity_date,loan_type,disbursed_amount,outstanding_balance,eir,days_past_due,...,building,land,bond,motor_vehicle,cash,equity,other,clean_report_date,clean_disbursement_date,clean_maturity_date
0,31/12/2022,1011000644003,1000644,43283,43648,PERSONAL_LOANS,4.800000e+01,4.800000e+01,0.0000,99,...,0.0,0.0,0,0.0,0.0,0,0.0,2022-12-31,2018-07-02,2019-07-02
1,31/12/2022,1011001359006,1001359,43283,43648,MCU,2.335890e+03,2.335890e+03,0.0000,99,...,0.0,0.0,0,0.0,0.0,0,0.0,2022-12-31,2018-07-02,2019-07-02
2,31/12/2022,1001001625002,1001625,41515,46073,MORTGAGE,1.018280e+06,1.778494e+06,0.0001,99,...,0.0,0.0,0,0.0,0.0,0,0.0,2022-12-31,2013-08-29,2026-02-20
3,31/12/2022,1001001625001,1001625,43982,44347,OVERDRAFTS,2.400000e-03,2.400000e-03,0.1400,99,...,0.0,0.0,0,0.0,0.0,0,0.0,2022-12-31,2020-05-31,2021-05-31
4,31/12/2022,1011001629010,1001629,44566,44931,OVERDRAFTS,4.000000e-03,4.000000e-03,0.1400,70,...,0.0,0.0,0,0.0,0.0,0,0.0,2022-12-31,2022-01-05,2023-01-05


In [4]:
import numpy_financial as npf
import pandas as pd


class ExposureAtDefault():

    def __init__(self, outstanding_balance, disbursement_date, maturity_date, dpd, stage, disbursed_amount, loan_type, eir, payment_frequency) -> None:
        self.outstanding_balance = outstanding_balance
        self.disbursement_date = disbursement_date
        self.maturity_date = maturity_date
        self.dpd = dpd
        self.stage = stage
        self.disbursed_amount = disbursed_amount
        self.loan_type = loan_type
        self.eir = eir
        self.payment_frequency = payment_frequency
    
    def periodic_rate(rate: float, periodicity: int) -> float:
        nth_rate = (1 + rate)**(1/periodicity) - 1
        return nth_rate

    @property
    def amortization(self, loan_amount: float, loan_duration: int, annual_rate: float, payment_frequency: int = 12) -> pd.DataFrame:
        """Create a loan amortization schedule for a given loan
    
        Parameters:
        - loan_amount: The outstanding loan balance
        - loan_tenure: The reamining expected time until fully repaid in years
        - annual_rate: The annual effective interest rate in decimal form e.g., 0.1 for 10%
        - payment_frequency: Interger representation of the frequency with which the loan is repaid i.e., total number of payments per year -> Default is monthly

        Returns:
        - amortization_schedule: DataFrame object containing the term structures for the Repayment Amount, Interest, Pricipal and Outstanding Balance
        
        """
       
        if self.payment_frequency not in set(range(0, 13)):
            raise ValueError("Payment Frequency must be integer value between 1 and 12")
        
        amortization_schedule = [loan_amount]
        principal_schedule = [0]
        interest_schedule = [0]
        payment_schedule = [0]
        payment = abs(round(npf.pmt(rate=self.periodic_rate(annual_rate, payment_frequency), nper=payment_frequency*loan_duration, pv=loan_amount), 2))
        counter = 1
        amount = loan_amount
        
        while round(amount, 0) > 0:

            if payment > amount:
                payment = round(amount * (1+self.periodic_rate(annual_rate, 12)), 2) + 0.001

            payment_schedule.append(payment) if not counter % int(12/payment_frequency) else payment_schedule.append(0)

            interest = round(amount * (self.periodic_rate(annual_rate, 12)), 2)
            interest_schedule.append(interest)

            principal = round(payment - interest, 2) if not counter % int(12/payment_frequency) else 0
            principal_schedule.append(principal)

            amount = round(amount * (1+self.periodic_rate(annual_rate, 12)) - payment, 2) if not counter % int(12/payment_frequency) else round(amount * (1+self.periodic_rate(annual_rate, 12)), 2)
            amortization_schedule.append(amount)

            counter +=1

        schedule_fin = pd.DataFrame({'Payment': payment_schedule,
                                    'Interest': interest_schedule,
                                    'Principal': principal_schedule,
                                    'Outstanding Balance': amortization_schedule})
        
        return schedule_fin
 

In [5]:
loan_book['remaining_time'] = loan_book['clean_maturity_date'] - loan_book['clean_report_date']
loan_book

,report_date,account_no,client_id,disbursement_date,maturity_date,loan_type,disbursed_amount,outstanding_balance,eir,days_past_due,...,land,bond,motor_vehicle,cash,equity,other,clean_report_date,clean_disbursement_date,clean_maturity_date,remaining_time
0,31/12/2022,1011000644003,1000644,43283,43648,PERSONAL_LOANS,4.800000e+01,4.800000e+01,0.0000,99,...,0.0,0,0.0,0.00,0,0.0,2022-12-31,2018-07-02,2019-07-02,-1278 days
1,31/12/2022,1011001359006,1001359,43283,43648,MCU,2.335890e+03,2.335890e+03,0.0000,99,...,0.0,0,0.0,0.00,0,0.0,2022-12-31,2018-07-02,2019-07-02,-1278 days
2,31/12/2022,1001001625002,1001625,41515,46073,MORTGAGE,1.018280e+06,1.778494e+06,0.0001,99,...,0.0,0,0.0,0.00,0,0.0,2022-12-31,2013-08-29,2026-02-20,1147 days
3,31/12/2022,1001001625001,1001625,43982,44347,OVERDRAFTS,2.400000e-03,2.400000e-03,0.1400,99,...,0.0,0,0.0,0.00,0,0.0,2022-12-31,2020-05-31,2021-05-31,-579 days
4,31/12/2022,1011001629010,1001629,44566,44931,OVERDRAFTS,4.000000e-03,4.000000e-03,0.1400,70,...,0.0,0,0.0,0.00,0,0.0,2022-12-31,2022-01-05,2023-01-05,5 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16549,31/12/2022,5031914950004,1914950,44925,47544,PERSONAL_LOANS,1.297000e+06,1.297000e+06,0.1300,20,...,0.0,0,0.0,1220424.26,0,0.0,2022-12-31,2022-12-30,2030-03-02,2618 days
16550,31/12/2022,3161915072004,1915072,44925,47909,PERSONAL_LOANS,7.710000e+05,7.710000e+05,0.1300,20,...,0.0,0,0.0,722377.14,0,0.0,2022-12-31,2022-12-30,2031-03-02,2983 days
16551,31/12/2022,4091915119004,1915119,44925,47909,PERSONAL_LOANS,3.330000e+05,3.330000e+05,0.1300,20,...,0.0,0,0.0,311999.47,0,0.0,2022-12-31,2022-12-30,2031-03-02,2983 days
16552,31/12/2022,11915188004,1915188,44925,47909,PERSONAL_LOANS,9.460000e+04,9.460000e+04,0.1300,20,...,0.0,0,0.0,88634.08,0,0.0,2022-12-31,2022-12-30,2031-03-02,2983 days


In [6]:
import math


def loan_duration(start_date: str, end_date: str) -> int:
    """Determine the loan tenure in months given the loan start and end dates
    
    Parameters:
    - start_date: Date string representing the loan start date
    - end_date: Date string representing the loan end date

    Returns:
    - tenure: the loan tenure in months as an integer

    """
    start = pd.to_datetime(start_date, dayfirst=False)
    end = pd.to_datetime(end_date, dayfirst=False)
    months = math.ceil((end - start).days / 365.25 * 12)
    return months

def months_between(start_date: str, end_date: str) -> int:
    """Determine the number of months between a given start and end date
    
    Parameters:
    - start_date: Date string representing the loan start date
    - end_date: Date string representing the loan end date

    Returns:
    - months: the number of months between start and end as an integer

    """
    start = pd.to_datetime(start_date, dayfirst=False)
    end = pd.to_datetime(end_date, dayfirst=False)
    tenure = math.ceil((end - start).days / 365.25 * 12)
    return tenure

In [7]:
amount = 1000000
annual_interest = 0.18

def periodic_rate(rate: float, periodicity: int) -> float:
    """Computes the periodic compound rate based on the input annual effective rate
    
    Parameters:
    - rate: annual effective interest rate
    - periodicity: the granularity to which to convert the annual rate to expressed as number of months per year i.e., 
    12 = monthly;
    4 = quarterly;
    2 = semi-annual;
    1 = annual  

    Returns:
    - nth_rate: the effective nth period rate
    
    """

    nth_rate = (1 + rate)**(1/periodicity) - 1
    return nth_rate

amortization_schedule = [amount]
principal_schedule = [0]
interest_schedule = [0]
payment_schedule = [0]
payment = npf.pmt(rate=periodic_rate(annual_interest, 12), nper=12, pv=1000000)
counter = 1
while round(amount, 0) > 0:
    
    payment_schedule.append(-round(payment, 2)) if not counter % 3 else payment_schedule.append(0)
    interest_schedule.append(round(amount * (periodic_rate(annual_interest, 12)), 2))
    principal_schedule.append(round(-payment - amount * (periodic_rate(annual_interest, 12)), 2))
    amount = round(amount * (1+periodic_rate(annual_interest, 12)), 2) if counter % 3 else round(amount * (1+periodic_rate(annual_interest, 12)) + payment, 2)
    amortization_schedule.append(amount)    
    counter +=1
    # print(amount)
schedule_fin = pd.DataFrame({'Payment': payment_schedule,
                             'Interest': interest_schedule,
                             'Principal': principal_schedule,
                             'Outstanding Balance': amortization_schedule})   

# amort_schedule = pd.DataFrame(amortization_schedule)
# checker = np.array(amortization_schedule)
schedule_fin
# len(interest_amort)

,Payment,Interest,Principal,Outstanding Balance
0,0.00,0.00,0.00,1000000.00
1,0.00,13888.43,77157.95,1013888.43
2,0.00,14081.32,76965.06,1027969.75
3,91046.38,14276.89,76769.49,951200.26
4,0.00,13210.68,77835.70,964410.94
5,0.00,13394.15,77652.22,977805.09
6,91046.38,13580.18,77466.20,900338.89
7,0.00,12504.29,78542.08,912843.18
8,0.00,12677.96,78368.42,925521.14
9,91046.38,12854.04,78192.34,847328.80


In [8]:
def loan_amortization_schedule(loan_amount: float, loan_duration: int, annual_rate: float, payment_frequency: int = 12) -> pd.DataFrame:
    """Create a loan amortization schedule for a given loan
    
    Parameters:
    - loan_amount: The outstanding loan balance
    - loan_tenure: The reamining expected time until fully repaid in years
    - annual_rate: The annual effective interest rate in decimal form e.g., 0.1 for 10%
    - payment_frequency: Interger representation of the frequency with which the loan is repaid i.e., total number of payments per year -> Default is monthly

    Returns:
    - amortization_schedule: DataFrame object containing the term structures for the Repayment Amount, Interest, Pricipal and Outstanding Balance
    
    """

    if payment_frequency not in set(range(0, 13)):
        raise ValueError("Payment Frequency must be integer value between 1 and 12")
    
    amortization_schedule = [loan_amount]
    principal_schedule = [0]
    interest_schedule = [0]
    payment_schedule = [0]
    payment = abs(round(npf.pmt(rate=periodic_rate(annual_rate, payment_frequency), nper=payment_frequency*loan_duration, pv=loan_amount), 2))
    counter = 1
    amount = loan_amount
    
    while round(amount, 0) > 0:

        if payment > amount:
            payment = round(amount * (1+periodic_rate(annual_rate, 12)), 2) + 0.001

        payment_schedule.append(payment) if not counter % int(12/payment_frequency) else payment_schedule.append(0)

        interest = round(amount * (periodic_rate(annual_rate, 12)), 2)
        interest_schedule.append(interest)

        principal = round(payment - interest, 2) if not counter % int(12/payment_frequency) else 0
        principal_schedule.append(principal)

        amount = round(amount * (1+periodic_rate(annual_rate, 12)) - payment, 2) if not counter % int(12/payment_frequency) else round(amount * (1+periodic_rate(annual_rate, 12)), 2)
        amortization_schedule.append(amount)

        counter +=1

    schedule_fin = pd.DataFrame({'Payment': payment_schedule,
                                'Interest': interest_schedule,
                                'Principal': principal_schedule,
                                'Outstanding Balance': amortization_schedule})
    
    return schedule_fin

loan_amortization_schedule(1000000, 2.5, 0.18, 2)

,Payment,Interest,Principal,Outstanding Balance
0,0.000,0.00,0.00,1000000.00
1,0.000,13888.43,0.00,1013888.43
2,0.000,14081.32,0.00,1027969.75
3,0.000,14276.89,0.00,1042246.64
4,0.000,14475.17,0.00,1056721.81
5,0.000,14676.21,0.00,1071398.02
6,254614.440,14880.04,239734.40,831663.62
7,0.000,11550.50,0.00,843214.12
8,0.000,11710.92,0.00,854925.04
9,0.000,11873.57,0.00,866798.61


In [9]:
def generate_series_geometric(n, N, common_ratio):
    # Solve for the first term (a) using the sum formula
    first_term = N / ((common_ratio**n - 1) / (common_ratio - 1))
    
    # Generate the series
    series = [first_term * common_ratio**i for i in range(n)]
    
    return series

# Example usage
n = 7  # Number of terms in the series
N = 115080  # Desired sum
common_ratio = 0.95  # Common ratio between terms

resulting_series = generate_series_geometric(n, N, common_ratio)
print(resulting_series)


[19074.283713203786, 18120.569527543597, 17214.541051166416, 16353.813998608093, 15536.123298677689, 14759.317133743802, 14021.351277056612]


In [10]:
import numpy as np
import matplotlib.pyplot as plt

def generate_non_outlier_numbers(n, total_sum, mean, std_dev):
    # Generate n random numbers from a normal distribution
    random_numbers = np.random.normal(loc=mean, scale=std_dev, size=n)

    # Ensure all numbers are positive
    random_numbers = np.abs(random_numbers)

    # Normalize the numbers to make their sum equal to total_sum
    normalized_numbers = random_numbers / np.sum(random_numbers)

    # Adjust the numbers to satisfy the sum constraint exactly
    random_numbers = normalized_numbers * total_sum

    return random_numbers

# Example usage
n = 7  # Specify the number of random numbers
total_sum = 115080  # Specify the desired sum
mean = 10  # Mean of the normal distribution
std_dev = 5  # Standard deviation of the normal distribution

random_numbers = generate_non_outlier_numbers(n, total_sum, mean, std_dev)

# Plot the distribution
print(random_numbers)
print(sum(random_numbers))


[15909.89579651 13112.95710712 20208.24170777 16100.58479022
 23151.75447138 16702.29806916  9894.26805783]
115079.99999999997
